<a href="https://colab.research.google.com/github/dineshgulati933/portneuf_IWRRI/blob/main/ee_python/Pneuf_OpenET_HUC12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#5/5/25: This script to fetch all OpenET models including ensemble for multiple years. It will apply reducer to each HUC12 feature
#to calculate mean, 25th, median, 75th in the region.
#The resulted csv for each year will be saved in drive which needs to be processed further to find WY totals or other operations.

In [1]:
import ee

In [2]:
# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize(project = 'ee-dineshgulati333')

In [9]:
# Define model collections (only 'et' band selected)
models = {
    'ensb': ee.ImageCollection("OpenET/ENSEMBLE/CONUS/GRIDMET/MONTHLY/v2_0").select('et_ensemble_mad'),
    'sebal': ee.ImageCollection("OpenET/GEESEBAL/CONUS/GRIDMET/MONTHLY/v2_0").select('et'),
    'sims': ee.ImageCollection("OpenET/SIMS/CONUS/GRIDMET/MONTHLY/v2_0").select('et'),
    'ptjpl': ee.ImageCollection("OpenET/PTJPL/CONUS/GRIDMET/MONTHLY/v2_0").select('et'),
    'alexi': ee.ImageCollection("OpenET/DISALEXI/CONUS/GRIDMET/MONTHLY/v2_0").select('et'),
    'sseb': ee.ImageCollection("OpenET/SSEBOP/CONUS/GRIDMET/MONTHLY/v2_0").select('et'),
    'metric': ee.ImageCollection("OpenET/EEMETRIC/CONUS/GRIDMET/MONTHLY/v2_0").select('et')
}

In [10]:
# Band rename dictionary
bandNamesDict = {
    'ensb': 'et_ensb',
    'sebal': 'et_sebal',
    'sims': 'et_sims',
    'ptjpl': 'et_ptjpl',
    'alexi': 'et_alexi',
    'sseb': 'et_sseb',
    'metric': 'et_metric'
}

In [11]:
aoi_box = ee.Geometry.BBox(-113.0208333332375, 41.9791666666035, -110.9791666665545, 44.0208333332865)
aoi = aoi_box.transform('EPSG:4269', 0.1)

wbd_huc12 = ee.FeatureCollection("USGS/WBD/2017/HUC12")
huc12_within_aoi = wbd_huc12.filter(ee.Filter.bounds(aoi))

huc12_bbox = huc12_within_aoi.geometry().bounds()

# Define the range of years and months
years = list(range(1999, 2024))  # 1999 to 2023
months = list(range(1, 13))  # January to December

# Get projection from an ET image to preserve scale
sample_projection = models['ensb'].first().projection()

In [12]:
# List to store tasks
tasks = []

# Outer loop over models
for model_name, model_et in models.items():

    for year in years:
        months_for_year = range(10, 13) if year == 1999 else months
        def create_monthly_composite(month):
            etm = model_et.filter(ee.Filter.calendarRange(year, year, 'year')) \
                          .filter(ee.Filter.calendarRange(month, month, 'month')) \
                          .filterBounds(huc12_bbox) \
                          .mosaic() \
                          .setDefaultProjection(sample_projection) \
                          .set('system:time_start', ee.Date.fromYMD(year, month, 1).millis()) \
                          .set('Year', year, 'Month', month, 'Date', ee.Date.fromYMD(year, month, 1).format('YYYY-MM-dd'))
            return etm

        allMonthlyComp = ee.ImageCollection([create_monthly_composite(m) for m in months_for_year])

        # Stats function → mean + percentiles only
        def Stats(image):
            reducer = ee.Reducer.mean() \
                .combine(ee.Reducer.percentile([25, 50, 75], ['25th', '50th', '75th']), sharedInputs=True)

            stats = image.reduceRegions(
                collection=huc12_within_aoi,
                reducer=reducer,
                scale=sample_projection.nominalScale() # or image.projection().nominalScale()
            )

            stats = stats.map(lambda f: f.set('Date', image.get('Date')))
            return stats

        et_stats = allMonthlyComp.map(Stats).flatten()

        # Selectors for export
        # selectors = ['Date', 'huc8',
        #              f'{band_name}_mean', f'{band_name}_25th', f'{band_name}_50th', f'{band_name}_75th']
        selectors = ['Date', 'huc12', 'mean', '25th', '50th', '75th']
        # Define export task
        task = ee.batch.Export.table.toDrive(
            collection=et_stats,
            description=f'OpenET_{model_name}_monthly_stat_{year}',
            folder='Pneuf_Open_ET_HUC12',
            fileFormat='CSV',
            selectors=selectors
        )

        # Start export
        task.start()
        tasks.append(task)
        print(f"Export started for {model_name}, {year}")

print("\nAll export tasks have been started in parallel!")


Export started for ensb, 1999
Export started for ensb, 2000
Export started for ensb, 2001
Export started for ensb, 2002
Export started for ensb, 2003
Export started for ensb, 2004
Export started for ensb, 2005
Export started for ensb, 2006
Export started for ensb, 2007
Export started for ensb, 2008
Export started for ensb, 2009
Export started for ensb, 2010
Export started for ensb, 2011
Export started for ensb, 2012
Export started for ensb, 2013
Export started for ensb, 2014
Export started for ensb, 2015
Export started for ensb, 2016
Export started for ensb, 2017
Export started for ensb, 2018
Export started for ensb, 2019
Export started for ensb, 2020
Export started for ensb, 2021
Export started for ensb, 2022
Export started for ensb, 2023
Export started for sebal, 1999
Export started for sebal, 2000
Export started for sebal, 2001
Export started for sebal, 2002
Export started for sebal, 2003
Export started for sebal, 2004
Export started for sebal, 2005
Export started for sebal, 2006
Ex